## Importing Pakakages 


In [1]:
!pip install -U transformers 
!pip install -U accelerate
!pip install -U datasets
!oio install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install --upgrade urllib3
!pip install py7zr

'oio' is not recognized as an internal or external command,
operable program or batch file.


##  Use the Amazon Fine Food Reviews dataset

In [2]:
import pandas as pd
# read csv file
reviews=pd.read_csv('/kaggle/input/assignment-4/Reviews.csv')

# drop everythin exept summry and text
reviews=reviews[['Summary','Text']]
reviews

# save csv file 


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/assignment-4/Reviews.csv'

In [ ]:
pip install -U nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Clean and preprocess the ‘Text’ and ‘Summary’ column from the dataset.

In [ ]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame(reviews)

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        # Convert text to lowercase
        return text.lower()
    else:
        return text  # Return the original value if it's not a string

# Apply preprocessing function to 'Summary' and 'Text' columns
df['Summary'] = df['Summary'].apply(preprocess_text)
df['Text'] = df['Text'].apply(preprocess_text)

# Drop rows where 'Summary' or 'Text' is NaN
df.dropna(subset=['Summary', 'Text'], inplace=True)

df


,Summary,Text
0,good quality dog food,i have bought several of the vitality canned d...
1,not as advertised,product arrived labeled as jumbo salted peanut...
2,"""delight"" says it all",this is a confection that has been around a fe...
3,cough medicine,if you are looking for the secret ingredient i...
4,great taffy,great taffy at a great price. there was a wid...
...,...,...
568449,will not do without,great for sesame chicken..this is a good if no...
568450,disappointed,i'm disappointed with the flavor. the chocolat...
568451,perfect for our maltipoo,"these stars are small, so you can give 10-15 o..."
568452,favorite training and reward treat,these are the best treats for training and rew...


In [ ]:
df['training'] = df['Text'] + 'TL;DR' + df['Summary'] 

In [3]:
df=df[['Summary','Text','training']][:1000]

NameError: name 'df' is not defined

## Model Training

## 1. Initialize a GPT-2 tokenizer and model from Hugging Face.

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset

# Initialize the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add padding token
tokenizer.pad_token = tokenizer.eos_token
tokenizer

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

## 2. Divide the dataset into training and testing (75:25)

In [ ]:
# divide data set iinto traning and testing (75:25)
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25)
train.shape, test.shape

((426320, 3), (142107, 3))

## 3.Implement a custom dataset class to prepare the data for training.

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self,tokenizer, df, max_length=100):
        self.max_length=max_length
        self.tokenizer = tokenizer
        self.eos = self
        self.eos_id=tokenizer.eos_token_id
        self.df=df
        self.output_text = []
        for i in self.df:
            tockenized_text=self.tokenizer.encode(df+self.eos)
            padding=self.pad_truncate(tockenized_text)
            self.output_text.append(padding)

        # Add padding token to the tokenizer
        self.tokenizer.pad_token = self.tokenizer.eos_token
    
    def __len__(self):
        return len(self.output_text)
    
    def __getitem__(self, index):
        return self.output_text[index]
        
    def pad_truncate(self, name):
        extralem=4
        name_len=len(name)-extralem
        if name_len<self.max_length:
            diff=self.max_length-name_len
            output_text=name+[self.eos_id]*diff
        elif name_len>self.max_length:
            output_text=name[:self.max_length+3]+[self.eos_id]
        else:
            output_text=name
        return output_text
            


In [ ]:
data_train = ReviewsDataset(train['Text'].tolist(), train['Summary'].tolist(), tokenizer)
data_test = ReviewsDataset(test['Text'].tolist(), test['Summary'].tolist(), tokenizer)
df_data=GptReviewDataset(tokenizer,df['training'].tolist(),max_length=100)

In [ ]:
for i in  if_data:
    print(tokenizer.decode(i),print(i))
    break

Note: you may need to restart the kernel to use updated packages.


In [4]:
from tourch.utils.data import DataLoader
data_load=DataLoader(df_data,batch_size=10,shuffle=True)

ModuleNotFoundError: No module named 'tourch'

In [ ]:
def trainmodel(model,optimizer,train_loader,epochs):
    model.train()
    for epoch in range(epochs):
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data=data.to(device)
            output=model(data,labels=data)
            loss=output.loss
            loss.backward()
            optimizer.step()
            if i%100==0:
                print(f'Epoch:{epoch}, Batch:{i}, Loss:{loss.item()}')

In [ ]:
from transformers import Trainer, TrainingArguments
import torch
import random

# Clear GPU cache
torch.cuda.empty_cache()

# Define the size of the subset you want to train on
subset_size = int(0.1 * len(data_train))  # 10% of the full dataset

# Randomly sample the subset from the full dataset
subset_indices = random.sample(range(len(data_train)), subset_size)
data_train_subset = torch.utils.data.Subset(data_train, subset_indices)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,
    gradient_accumulation_steps=4,  # Increase accumulation to reduce memory usage
    evaluation_strategy="epoch",    # Evaluate at the end of each epoch
    save_strategy="epoch",          # Save at the end of each epoch
    load_best_model_at_end=True,    # Load the best model when finished training
    metric_for_best_model="accuracy",# Use accuracy to determine the best model
    greater_is_better=True,         # Higher accuracy is better
    save_total_limit=1              # Limit the total number of checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train_subset,
    eval_dataset=data_test
)

# Start training
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 42.12 MiB is free. Process 2275 has 15.85 GiB memory in use. Of the allocated memory 15.48 GiB is allocated by PyTorch, and 86.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from transformers import Trainer, TrainingArguments
import torch
import random
from rouge import Rouge

# Clear GPU cache
torch.cuda.empty_cache()

# Define the size of the subset you want to train on
subset_size = int(0.1 * len(data_train))  # 10% of the full dataset

# Randomly sample the subset from the full dataset
subset_indices = random.sample(range(len(data_train)), subset_size)
data_train_subset = torch.utils.data.Subset(data_train, subset_indices)

# Hyperparameter grid
learning_rates = [5e-5, 3e-5, 2e-5]
batch_sizes = [4, 8]
epochs = [2, 3]

best_rouge_score = 0.0
best_hyperparams = {}

# Evaluation function
def compute_rouge(predictions, references):
    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return scores

for lr in learning_rates:
    for batch_size in batch_sizes:
        for epoch in epochs:
            # Clear GPU cache
            torch.cuda.empty_cache()

            # Training arguments
            training_args = TrainingArguments(
                output_dir='./results',
                num_train_epochs=epoch,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=64,
                warmup_steps=500,
                weight_decay=0.01,
                logging_dir='./logs',
                logging_steps=10,
                learning_rate=lr,
                gradient_accumulation_steps=4,
                evaluation_strategy="epoch",
                save_strategy="epoch",
                load_best_model_at_end=True,
                metric_for_best_model="accuracy",
                greater_is_better=True,
                save_total_limit=1
            )

            # Initialize Trainer
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=data_train_subset,
                eval_dataset=data_test,
                tokenizer=tokenizer
            )

            # Start training
            trainer.train()

            # Generate predictions
            predictions = trainer.predict(test_dataset=data_test)
            predicted_summaries = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

            # Compute ROUGE scores
            rouge_scores = compute_rouge(predicted_summaries, actual_summaries)

            # Print and save the best hyperparameters
            if rouge_scores['rouge-1']['f'] > best_rouge_score:
                best_rouge_score = rouge_scores['rouge-1']['f']
                best_hyperparams = {'lr': lr, 'batch_size': batch_size, 'epochs': epoch}

# Print the best hyperparameters and ROUGE scores
print(f"Best ROUGE-1 F1-Score: {best_rouge_score}")
print(f"Best Hyperparameters: {best_hyperparams}")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from rouge import Rouge
from transformers import GPT2Tokenizer

# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# Generate predictions on the test set
predictions = trainer.predict(test_dataset=data_test)
predicted_summaries = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Compute ROUGE scores
def compute_rouge(predictions, references):
    rouge = Rouge()
    scores = rouge.get_scores(predictions, references, avg=True)
    return scores

# Assuming 'actual_summaries' is a list of actual summaries from the test set
actual_summaries = data_test['summary']  # Replace with your actual test set summaries

rouge_scores = compute_rouge(predicted_summaries, actual_summaries)

# Print ROUGE scores
print("ROUGE-1: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
    rouge_scores['rouge-1']['p'], rouge_scores['rouge-1']['r'], rouge_scores['rouge-1']['f']))
print("ROUGE-2: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
    rouge_scores['rouge-2']['p'], rouge_scores['rouge-2']['r'], rouge_scores['rouge-2']['f']))
print("ROUGE-L: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(
    rouge_scores['rouge-l']['p'], rouge_scores['rouge-l']['r'], rouge_scores['rouge-l']['f']))


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 42.12 MiB is free. Process 2275 has 15.85 GiB memory in use. Of the allocated memory 15.48 GiB is allocated by PyTorch, and 85.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF